In [5]:
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np
from scipy import sparse 
import sys
from gaussiandata import SyntheticTarget
from onlinePrototypes import protonline
from onlineCriticism import crionline
#import greedy_functions
import math
import scipy

In [6]:
SEED = 666
number_of_gaussians = 2
ex = SyntheticTarget(seed=SEED, dist_components=number_of_gaussians)

N = 1000
M = 10
data = ex.draw_sample(N)
print  data.size
print data[(2,3,4),:]
gamma = 2
kernel_matrix = rbf_kernel(data, gamma = gamma)

2000
[[ 2.58801945 -4.10201855]
 [-1.1745815  -3.81182871]
 [ 3.41529632 -3.59914177]]


In [7]:
def rbf(x,y, gamma):
    x = np.asarray(x)
    y = np.asarray(y)
    return scipy.exp(-gamma*np.linalg.norm(x-y))

def our_rbf_kernel(current_kernel_subset,  gamma, x_new, n, M, data, current_indices):
    print "we are here"
    new_row = np.zeros(n+1)
    new_col = np.zeros(M)
    for i in range(n+1):
        new_row[i] = rbf(x = x_new, y = data[i], gamma = gamma)
         
    new_col = new_row[current_indices]
    new_row = np.matrix(new_row)
    new_col = np.matrix(new_col)

    new_kernel_subset = np.hstack((current_kernel_subset, new_col.T))
    new_kernel_subset = np.vstack((new_kernel_subset, new_row))
    
    return new_kernel_subset

In [8]:
gamma = 2
prototype_indices = np.arange(M) #assign first M points as prototypes. CONSIDER another method later.
criticism_indices = np.arange(M, 2*M) # assign second M points as prototypes.

#calculate the initial kernel matrix for the first M points
#current_kernel_matrix_proto = rbf_kernel(data[prototype_indices,:], gamma=gamma)


#current_kernel_matrix_criticism = rbf_kernel(data[criticism_indices,:], gamma = gamma) #to check with Petya

current_kernel_matrix = rbf_kernel(data[range(2*M)], gamma = gamma)
current_kernel_matrix_proto = current_kernel_matrix[:M]
current_kernel_matrix_criticism = current_kernel_matrix[range(M, 2*M)]


for i in range(2*M, N):
    
    new_point_index = i # assign the new data point index
    accept = 1 # decide whether to skip the criticism part or not 
    
    #prototype part
    if accept == 1:
        #add new column and row to the kernel matrix for the new point; 
        #kernel matrix is always growing
        #current_kernel_subset,  gamma, x_new, n, timeseries):
        
        print "here"
        print "i"
        print i

        new_kernel_matrix_proto = our_rbf_kernel(current_kernel_subset = current_kernel_matrix_proto, gamma = gamma,  
                                                 x_new= data[i], n = i, M = M,  data = data[range(i+1)], 
                                                 current_indices = prototype_indices)

        #run the greedy algorithm on these M+1 data points to select M prototypes
        print prototype_indices
        prototype_indices_temp = protonline(new_kernel_matrix_proto, 
                                                  selectedprotos = prototype_indices, m = M)
        print "prototype_indices_temp"
        print type(prototype_indices_temp)
        print prototype_indices_temp
        print "prototype_indices"
        print type(prototype_indices)
        print prototype_indices
        if (prototype_indices_temp == prototype_indices).all():
             accept = 0
        
        else:
            print "ciao"
            prototype_indices = prototype_indices_temp # update the indices
            current_kernel_matrix_proto = new_kernel_matrix_proto # update the kernel matrix
    
    
    #criticism part
    if accept == 0:
        
        new_kernel_matrix_criticism = our_rbf_kernel(current_kernel_subset = current_kernel_matrix_criticism, 
                                                     gamma = gamma,  x_new= data[i], n = i, M = M, 
                                                     data = data[range(i+1)], current_indices = criticism_indices)
        
        #run the greedy algorithm on these M+1 data points to select M criticism
        criticism_indices_temp = crionline(new_kernel_matrix_criticism, 
                                                  selectedprotos = prototype_indices, selectedcritic = criticism_indices, c = M, reg='logdet')

        
        if (criticism_indices_temp == criticism_indices).all():
            accept == 0
        
        else:
            criticism_indices = criticism_indices_temp # update the indices
            current_kernel_matrix_criticism = new_kernel_matrix_criticism # update the kernel matrix
    
    

here
i
20
we are here
[0 1 2 3 4 5 6 7 8 9]
we are out of the if-else
i
0
s1 array
[ 0.2337025   0.20655686  0.23959615  0.61542     0.2905379   0.38624166
  0.44004073  0.50677863  0.68791655  0.26476668  0.22448578]
candidates
[ 0  1  2  3  4  5  6  7  8  9 10]
argmax
8
selected
[8]
candidates2
[ 0  1  2  3  4  5  6  7  8  9 20]
(11,)
<type 'numpy.ndarray'>
we are out of the if-else
i
1
s1 array
[-0.3239226  -0.29344314 -0.26040385 -0.75200846 -0.2094621  -0.51717967
 -0.34840798 -0.21338733 -0.23523332 -0.27556314]
candidates
[ 0  1  2  3  4  5  6  7  9 10]
argmax
4
selected
[8 4]
candidates2
[ 0  1  2  3  4  5  6  7  8  9 20]
(11,)
<type 'numpy.ndarray'>
we are out of the if-else
i
2
s1 array
[-0.13804756 -0.59209488 -0.19602045 -0.29619897 -0.21603923 -0.08559174
  0.02666799 -0.37672627 -0.27485925]
candidates
[ 0  1  2  3  5  6  7  9 10]
argmax
7
selected
[8 4 7]
candidates2
[ 0  1  2  3  4  5  6  7  8  9 20]
(11,)
<type 'numpy.ndarray'>
we are out of the if-else
i
3
s1 array
[-

ValueError: all the input array dimensions except for the concatenation axis must match exactly